In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from __future__ import division
from nltk import sent_tokenize, word_tokenize
from wikipedia import page

In [30]:
data = pd.read_csv("user_question.csv", header=None)
data.columns = ['question','group']

In [31]:
train = data['question'].values.tolist()
label = data['group'].values.tolist()

In [32]:
corpus = train + label

In [35]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [36]:
tok_corp = [nltk.word_tokenize(sent.decode('utf-8')) for sent in corpus]

In [37]:
model = Word2Vec(tok_corp, size=50, window=5, min_count=3, workers=2)

In [45]:
model.most_similar('How')

[(u'curl', 0.2829551100730896),
 (u'want', 0.14266327023506165),
 (u'What', 0.13052792847156525),
 (u'to', 0.11642688512802124),
 (u'Shoulders', 0.08410151302814484),
 (u'abs', 0.05228054150938988),
 (u'Triceps', 0.01863226667046547),
 (u'I', 0.006797999143600464),
 (u'do', -0.019749730825424194),
 (u'Leg', -0.023615125566720963)]

In [51]:
wikipage = page("Physical fitness")

In [53]:
sentences = [word_tokenize(sent) for sent in sent_tokenize(wikipage.content)]

In [56]:
len(sentences)

137

In [57]:
wiki_model = Word2Vec(sentences, min_count=2, size=50, window=5)  

In [58]:
list(wiki_model.wv.vocab.keys())[:10]

[u'all',
 u'resistance',
 u'consists',
 u'to',
 u'must',
 u'program',
 u'helps',
 u'include',
 u'activities',
 u'Training']

In [62]:
wiki_model.most_similar('resistance')

[(u'in', 0.6016296148300171),
 (u'.', 0.5888518691062927),
 (u'such', 0.5855332016944885),
 (u'Cancer', 0.5716492533683777),
 (u'physical', 0.5606939792633057),
 (u'body', 0.5583063364028931),
 (u'work', 0.5521804094314575),
 (u'have', 0.5517875552177429),
 (u'risk', 0.5498833656311035),
 (u'recommendations', 0.534516453742981)]

In [64]:
exrx_data = pd.read_json('fit_modified_all.json', orient = 'records', dtype={"A":str, "B":list})
exrx_train = exrx_data[['exercisename','preparation','comments','execution']]

In [65]:
exrx_train.head()

,exercisename,preparation,comments,execution
0,Safety Barbell Standing Leg Calf Raise,Stand facing safety barbell on rack upper ches...,Position rack just below lowest range of motio...,Raise heels by extending ankles as high as pos...
1,Cable Triceps Extension (with rope),"From low pulley cable, grasp ends of rope atta...",Let cable attachment pull arm back to maintain...,Raise ends of rope overhead by extending forea...
2,Safety Bar Seated Calf Raise,Place safety bar on rack approximately lower l...,Slide forward to edge of bench and rack safety...,Lower heels by bending ankles until calves are...
3,Sled 45° Reverse Calf Raise (plate loaded),Sit on machine with lower back against padding...,Keep knees straight throughout exercise. Also ...,Pull forefoot of both feet up and back toward ...
4,Sled 45° Reverse Calf Raise,Sit on machine with lower back against padding...,Keep knees straight throughout exercise. Also ...,Pull forefoot of both feet up and back toward ...


In [69]:
corpus = exrx_train['exercisename'].values.tolist() + exrx_train['preparation'].values.tolist() + exrx_train['comments'].values.tolist()+exrx_train['execution'].values.tolist()  

In [70]:
len(corpus)

2548

In [71]:
corpus[:50]

[u'Safety Barbell Standing Leg Calf Raise',
 u'Cable Triceps Extension (with rope)',
 u'Safety Bar Seated Calf Raise',
 u'Sled 45\xb0 Reverse Calf Raise (plate loaded)',
 u'Sled 45\xb0 Reverse Calf Raise',
 u'Sled 45\xb0 Reverse Calf Press',
 u'Lever Seated Reverse Calf Press',
 u'Lever Reverse Calf Raise',
 u'Lever Donkey Reverse Calf Raise',
 u'Sled Standing Reverse Calf Raise',
 u'Dumbbell Seated Front Lateral Raise',
 u'Dumbbell Kickback',
 u'Dumbbell Front Lateral Raise',
 u'Sled Hack Calf Press',
 u'Sled Donkey Calf Raise',
 u'Dumbbell Full Can Lateral Raise',
 u'Weighted Glute-Ham Raise',
 u'Suspended Hyperextension',
 u'Cable Hip Adduction',
 u'Dumbbell Shoulder Internal Rotation (on bench)',
 u'Cable Standing Shoulder Internal Rotation',
 u'Lever Upright Shoulder Internal Rotation',
 u'Cable Seated Hip External Rotation',
 u'Cable Lying Hip External Rotation',
 u'Suspended Shoulder External Rotation',
 u'Suspended Pike',
 u'Pike on Discs',
 u'Pike on Power Wheel',
 u'Lying Sci